# ADA Homework 2

In [60]:
# Import libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [76]:
base_URL = 'https://www.topuniversities.com'

In [77]:
false_URL = '/university-rankings/world-university-rankings/2018'
rank_URL = '/sites/default/files/qs-rankings-data/357051.txt'

-----------------------

This part shows how we first tried to treat the data but realised that the data was not retrievable from the `false_url`

In [78]:
# Make the request
r = requests.get(base_URL + false_URL)
body = r.text

In [64]:
# Extract the header
soup = BeautifulSoup(body, 'html.parser')
soup.h1

In [65]:
links = soup.find_all('a')
interesting_links = []

In [66]:
for link in links:
    if (not link.get('href') is None and link.get('href').startswith('/universities')):
        interesting_links.append(link)

interesting_links

[]

In [67]:
links

[]

------------

In [82]:
r = requests.get(base_URL + rank_URL)

In [83]:
type(r.json())

dict

In [88]:
json = r.json()
data = json['data'][:200]

In [261]:
def get_info(url):
    req = requests.get(base_URL + url)
    body = req.text
    soup = BeautifulSoup(body, 'html.parser')
    return soup.findAll('div', {'class': 'number'})

In [262]:
def parse_int(s):
    return int(''.join(c for c in s if c.isdigit()))

In [288]:
def put_names(l):
    d = {}
    names = ['staff total', 'staff international', 'students total', 'students international']
    for i in range(4):
        if i < len(l):
            d[names[i]] = l[i]
        else:
            d[names[i]] = 0
    return d

In [289]:
def treat(url):
    numbers = [parse_int(div.text) for div in get_info(url)][:4]
    return put_names(numbers)

In [290]:
useful_data = []
for uni in data:
    useful_data.append({'name': uni['title'], 'country': uni['country'], 'region': uni['region'],'rank': uni['rank_display'], 'url': uni['url']})

In [291]:
useful_data[:5]

[{'country': 'United States',
  'name': 'Massachusetts Institute of Technology (MIT)',
  'rank': '1',
  'region': 'North America',
  'url': '/universities/massachusetts-institute-technology-mit'},
 {'country': 'United States',
  'name': 'Stanford University',
  'rank': '2',
  'region': 'North America',
  'url': '/universities/stanford-university'},
 {'country': 'United States',
  'name': 'Harvard University',
  'rank': '3',
  'region': 'North America',
  'url': '/universities/harvard-university'},
 {'country': 'United States',
  'name': 'California Institute of Technology (Caltech)',
  'rank': '4',
  'region': 'North America',
  'url': '/universities/california-institute-technology-caltech'},
 {'country': 'United Kingdom',
  'name': 'University of Cambridge',
  'rank': '5',
  'region': 'Europe',
  'url': '/universities/university-cambridge'}]

In [292]:
cop = useful_data.copy()

In [293]:
treat(cop[0]['url'])

{'staff international': 1679,
 'staff total': 2982,
 'students international': 3717,
 'students total': 11067}

In [294]:
cop[0].update(treat(cop[0]['url']))

In [295]:
cop[0]

{'country': 'United States',
 'name': 'Massachusetts Institute of Technology (MIT)',
 'rank': '1',
 'region': 'North America',
 'staff international': 1679,
 'staff total': 2982,
 'students international': 3717,
 'students total': 11067,
 'url': '/universities/massachusetts-institute-technology-mit'}

__Dislaimer:__ This takes quite some time as it opens 200 connections (one for each university in the table)

In [296]:
for i in range(len(useful_data)):
    useful_data[i].update(treat(useful_data[i]['url']))

In [297]:
ranking_df = pd.DataFrame(useful_data)
ranking_df.drop('url', axis=1).head(12)

,country,name,rank,region,staff international,staff total,students international,students total
0,United States,Massachusetts Institute of Technology (MIT),1,North America,1679,2982,3717,11067
1,United States,Stanford University,2,North America,2042,4285,3611,15878
2,United States,Harvard University,3,North America,1311,4350,5266,22429
3,United States,California Institute of Technology (Caltech),4,North America,350,953,647,2255
4,United Kingdom,University of Cambridge,5,Europe,2278,5490,6699,18770
5,United Kingdom,University of Oxford,6,Europe,2964,6750,7353,19720
6,United Kingdom,UCL (University College London),7,Europe,2554,6345,14854,31080
7,United Kingdom,Imperial College London,8,Europe,2071,3930,8746,16090
8,United States,University of Chicago,9,North America,635,2449,3379,13557
9,Switzerland,ETH Zurich - Swiss Federal Institute of Techno...,10,Europe,1886,2477,7563,19815
